In [1]:
import h5py
import numpy as np
import pandas as pd 
import scipy

C:\Users\byan\AppData\Local\Temp\ipykernel_12252\2735699808.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def read_data(path_of_data):
  """
  read data seaved in h5py format
  assuming group keys that exists are X,y 
  """
  with h5py.File(path_of_data, "r") as f:
    X,y = f["X"][:],f['y'][:]
  return X,y


def signals_log_sum(signal):
    from scipy.special import logsumexp
    logSum=logsumexp(signal,-2)
    return logSum

def signals_rms(signals):
  rms=np.sqrt(np.mean(signals**2,axis=-2))
  return rms 

def signals_std(signals):
  std = np.std(signals,axis=-2)
  return std

def signals_avg_power(signals):
  power=np.mean(signals,axis=-2)
  return power

def signals_mean_abs(signals):
  mean_abs=np.mean(np.abs(signals),axis=-2)
  return mean_abs

def signals_energy(signals):
  energy=np.mean(np.square(np.abs(signals)),axis=-2)
  return energy

def band_pass(record, l=0.5, h=60):
    """
    Apply a band-pass filter to multi-channel time series data.

    Args:
        record (np.ndarray): Multi-channel time series data with shape (n_signals, Time, N_channels).
        l (float): Lower frequency cutoff for the band-pass filter.
        h (float): Upper frequency cutoff for the band-pass filter.

    Returns:
        np.ndarray: Filtered channel data with the same shape as the input record.
    """
    sos = scipy.signal.butter(4, (l, h), 'bandpass', fs=256, output='sos')
    filtered_channel = scipy.signal.sosfilt(sos, record, axis=-2)
    return filtered_channel
 

def get_features(dataset,normalized=True):
  features=np.concatenate([
                               signals_log_sum(dataset),signals_avg_power(dataset),
                               signals_energy(dataset),signals_rms(dataset) ,
                               signals_mean_abs(dataset),signals_std(dataset)
],axis=-1)
  
  return features 

In [3]:
train_18_X,train_18_y= read_data("data/train.hdf5")
val_18_X,val_18_y=read_data("data/test.hdf5")

In [4]:
print(train_18_X.shape,train_18_y.shape)
print(val_18_X.shape,val_18_y.shape)

(15794, 3840, 18) (15794,)
(1566, 3840, 18) (1566,)


In [5]:
#nans_indicies= np.any(np.isnan(train_18_X),axis=(1,2))
#train_18_X,train_18_y = train_18_X[~nans_indicies],train_18_y[~nans_indicies]
#nans_indicies= np.any(np.isnan(val_18_X),axis=(1,2))
#val_18_X,val_18_y = val_18_X[~nans_indicies],val_18_y[~nans_indicies]

In [5]:
print(train_18_X.shape,train_18_y.shape)
print(val_18_y.shape,val_18_y.shape)

(15794, 3840, 18) (15794,)
(1566,) (1566,)


In [6]:
train_18_X = get_features(train_18_X,)
val_18_X = get_features(val_18_X,)

In [8]:
print(train_18_X.shape,train_18_y.shape)
print(val_18_X.shape,val_18_y.shape)

(15794, 108) (15794,)
(15794, 108) (1566,)


In [14]:
train_18_y = np.where(train_18_y==1,'seizure','normal')
val_18_y = np.where(val_18_y==1,'seizure','normal')

In [15]:
train_features_df = pd.DataFrame(np.hstack((train_18_X,train_18_y[:,np.newaxis])))
test_features_df = pd.DataFrame(np.hstack((val_18_X,val_18_y[:,np.newaxis])))


In [16]:
train_features_df.to_csv('data/features_train.csv',index=False)
test_features_df.to_csv('data/features_test.csv',index=False)

In [11]:
train_features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
0,325.513611,624.963501,409.708496,603.394531,413.249847,353.015900,362.864746,681.411255,439.365082,424.517731,...,149.210709,137.938599,151.988678,123.457977,131.850677,111.988602,98.615273,114.103111,95.453064,1.0
1,205.108353,131.942551,136.784683,158.648376,151.120956,119.375191,128.278107,128.167191,134.984467,125.182358,...,36.867420,50.237579,60.317097,46.421406,39.446640,58.723095,56.553867,59.245579,51.779766,0.0
2,146.038895,123.386017,89.021904,184.802277,70.034050,114.691467,166.461548,82.594673,147.337189,128.404160,...,44.273838,21.897675,31.232632,18.562162,26.380524,28.410452,37.975960,22.493172,33.861504,0.0
3,703.291565,1210.647095,1143.052490,1150.476196,1290.354126,1089.133911,1259.096436,1345.836426,1123.516479,1070.755371,...,352.153687,369.538147,378.030884,354.959656,228.551407,317.236786,294.713989,391.028198,349.547455,1.0
4,116.763390,92.539642,108.608383,169.078156,131.478729,233.517838,114.440117,110.793312,107.273300,108.749878,...,34.908272,41.919285,37.220177,22.589159,28.194540,42.935658,32.492867,32.227161,54.085892,0.0
